# SHA-512

In [5]:
from utils import *
from config import *
from convertEverything import *

## Step 1: ADD PADDING

In [6]:
# Practice
# 1. Initial Hash Values
message = 'abc'


In [7]:

L = len(convert_to_bin(message))
print(convert_to_bin(message))
print(f"Number bits of message: {L}")
print(convert_to_hex(message))

011000010110001001100011
Number bits of message: 24
616263


In [8]:
padding_message = add_padding(message, N)
print(len(add_padding(message, N)))
print(len(add_padding(message, N, 'bin')))
# type(padding_message)

256
1024


## Step 2: INITIAL HASH VALUES (IV)

In [9]:
# from the mpmath library
# print(mp) # precision
mp.dps = 50
int(fmul(frac(sqrt(2)), pow(2, 64)))
print(hex(int(fmul(frac(sqrt(2)), pow(2, 64)))))

0x6a09e667f3bcc908


In [10]:
s.sieve[1] # the first prime
first_eight_primes = s.sieve[1:9] # the first 8 primes

In [12]:
bin_fraction_8primes = [sqrt_64bits_hex(i) for i in first_eight_primes]
bin_fraction_8primes


['6a09e667f3bcc908',
 'bb67ae8584caa73b',
 '3c6ef372fe94f82b',
 'a54ff53a5f1d36f1',
 '510e527fade682d1',
 '9b05688c2b3e6c1f',
 '1f83d9abfb41bd6b',
 '5be0cd19137e2179']

In [13]:
initial_hash = [
    0x6a09e667f3bcc908,
    0xbb67ae8584caa73b,
    0x3c6ef372fe94f82b,
    0xa54ff53a5f1d36f1,
    0x510e527fade682d1,
    0x9b05688c2b3e6c1f,
    0x1f83d9abfb41bd6b,
    0x5be0cd19137e2179,
]
[hex(i)[2:] for i in initial_hash]


['6a09e667f3bcc908',
 'bb67ae8584caa73b',
 '3c6ef372fe94f82b',
 'a54ff53a5f1d36f1',
 '510e527fade682d1',
 '9b05688c2b3e6c1f',
 '1f83d9abfb41bd6b',
 '5be0cd19137e2179']

In [16]:
initial_hash = {
    'a': 0x6a09e667f3bcc908,
    'b': 0xbb67ae8584caa73b,
    'c': 0x3c6ef372fe94f82b,
    'd': 0xa54ff53a5f1d36f1,
    'e': 0x510e527fade682d1,
    'f': 0x9b05688c2b3e6c1f,
    'g': 0x1f83d9abfb41bd6b,
    'h': 0x5be0cd19137e2179,
}
initial_hash_hex = {k: hex(v)[2:] for k, v in initial_hash.items()}
print(initial_hash_hex['a'])
print(initial_hash['a'])

6a09e667f3bcc908
7640891576956012808


## Step 3: 
Process message in 1024 bits (128 byte) blocks: The heart of the algorithm is a module that consists of 80 rounds; this module is labeled F in Figure 11.9. The logic is illustrated in Figure 11.10. 

Each round takes as input the 512-bit buffer value, abcdefgh, and updates the contents of the buffer. At input to the first round, the buffer has the value of the intermediate hash value, Hi - 1 . 

Each round t makes use of a 64-bit value Wt, derived from the current 1024-bit block being processed (Mi). These values are derived using a message schedule described subsequently. Each round also makes use of an additive constant Kt, where 0 … t … 79 indicates one of the 80 rounds. These words represent the first 64 bits of the fractional parts of the cube roots of the first 80 prime numbers. The constants provide a “randomized” set of 64-bit patterns, which should eliminate any regularities in the input data. Table 11.4 shows these constants in hexadecimal format (from left to right)



# <div style="display:flex; justify-content: space-around"> <img src="./images/img.png" width="400" height="400"> <img src="./images/img_1.png" width="400" height="400"> </div>

In [17]:
# Practice
def SHA512(message):
    for i in range(0, len(message), 16):
        block = message[i:i+16]
        print(block)
        for i in range(0, 80):
            break
SHA512(padding_message)


6162638000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000000
0000000000000018
